In [1]:
import json
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import time

# Manipulate Page

In [2]:
driver_location = "/Users/davidmanspeaker/Downloads/chromedriver"
driver = webdriver.Chrome(executable_path=driver_location)
driver.get("http://www.google.com")
driver.find_element_by_class_name('gb_Ne').click()
inputElement = driver.find_element_by_id("Email")
inputElement.send_keys('david4cider@gmail.com')
inputElement.send_keys(Keys.ENTER)

time.sleep(3)

inputElement = driver.find_element_by_id("Passwd")
inputElement.send_keys('Lifeisgood!')
inputElement.send_keys(Keys.ENTER)

# Set Up Google Scrape

In [3]:
%system GoogleScraper -m http -p 100 -n 50 -q "should site:quora.com" --output-filename quora1.json

['2016-09-14 20:04:03,426 - GoogleScraper.caching - INFO - 2521 cache files found in .scrapecache/',
 '2016-09-14 20:04:03,427 - GoogleScraper.caching - INFO - 100/100 objects have been read from the cache. 0 remain to get scraped.']

# Open the file and Get Results

In [4]:
with open("quora1.json", 'r') as f:
    data  = json.load(f)
    
all_results = []
for d in data:
    all_results.extend(d['results'])

# Inspect html

In [8]:
questions = []
for result in all_results:
    link = result['link']
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    question_area = soup.find("div", { "class" : "QuestionArea" })
    question_span = question_area.find("span", {"class": "rendered_qtext"})
    answers = soup.find("div", {"class", "QuestionPageAnswerHeader"})
    answers_num = answers.text.split()[0]
    follows_num = soup.find("span", {"class", "count"}).text
    questions.append([question_span.text, follows_num, answers_num])

AttributeError: 'NoneType' object has no attribute 'find'

# Output Results as CSV

In [7]:
df_questions = pd.DataFrame(questions, columns=['Question', 'Follows', 'Answers'])
df_questions['Follows'] = df_questions['Follows'].map(lambda x: int(float(x[:-1]) * 1000) if x[-1] == 'k' else int(x))
df_questions['Answers'] = df_questions['Answers'].map(lambda x: 100 if '+' in x else int(x))
pd.options.display.max_colwidth = 150
df_questions.sort_values('Follows', ascending=False, inplace=True)
df_questions.head(20)
df_questions.to_csv('/Users/davidmanspeaker/GitHub/WebScrape/Yahoo! Answers/Results/Quora Scrape Results_Should' + time.strftime("%Y%m%d-%H%M%S") + '.csv', index=False)